# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 9 2023 9:38AM,258996,15,30014616,"Alliance Pharma, Inc.",Released
1,Mar 9 2023 9:38AM,258996,15,30014617,"Alliance Pharma, Inc.",Released
2,Mar 9 2023 9:38AM,258996,15,30014610,"Alliance Pharma, Inc.",Released
3,Mar 9 2023 9:38AM,258996,15,30014611,"Alliance Pharma, Inc.",Released
4,Mar 9 2023 9:38AM,258996,15,30014612,"Alliance Pharma, Inc.",Released
5,Mar 9 2023 9:38AM,258996,15,30014613,"Alliance Pharma, Inc.",Released
6,Mar 9 2023 9:38AM,258996,15,30014614,"Alliance Pharma, Inc.",Released
7,Mar 9 2023 9:38AM,258996,15,30014615,"Alliance Pharma, Inc.",Released
8,Mar 9 2023 9:37AM,258995,10,PRF-55042,Bio-PRF,Released
9,Mar 9 2023 9:37AM,258995,10,PRF-55048,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,258990,Released,1
15,258993,Released,1
16,258994,Released,1
17,258995,Released,15
18,258996,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258990,NaN,1.0
258993,NaN,1.0
258994,NaN,1.0
258995,NaN,15.0
258996,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258866,0.0,17.0
258947,0.0,36.0
258960,0.0,1.0
258964,0.0,7.0
258975,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258866,0,17
258947,0,36
258960,0,1
258964,0,7
258975,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258866,0,17
1,258947,0,36
2,258960,0,1
3,258964,0,7
4,258975,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258866,,17
1,258947,,36
2,258960,,1
3,258964,,7
4,258975,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 9 2023 9:38AM,258996,15,"Alliance Pharma, Inc."
8,Mar 9 2023 9:37AM,258995,10,Bio-PRF
23,Mar 9 2023 9:18AM,258994,10,Eywa Pharma Inc.
24,Mar 9 2023 9:18AM,258947,15,"Alliance Pharma, Inc."
60,Mar 9 2023 9:17AM,258866,15,"Brookfield Pharmaceuticals, LLC"
77,Mar 9 2023 9:16AM,258993,22,"NBTY Global, Inc."
78,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation
79,Mar 9 2023 9:01AM,258989,10,Methapharm-G
80,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc."
81,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 9 2023 9:38AM,258996,15,"Alliance Pharma, Inc.",,8
1,Mar 9 2023 9:37AM,258995,10,Bio-PRF,,15
2,Mar 9 2023 9:18AM,258994,10,Eywa Pharma Inc.,,1
3,Mar 9 2023 9:18AM,258947,15,"Alliance Pharma, Inc.",,36
4,Mar 9 2023 9:17AM,258866,15,"Brookfield Pharmaceuticals, LLC",,17
5,Mar 9 2023 9:16AM,258993,22,"NBTY Global, Inc.",,1
6,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,,1
7,Mar 9 2023 9:01AM,258989,10,Methapharm-G,,1
8,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",,1
9,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:38AM,258996,15,"Alliance Pharma, Inc.",8,
1,Mar 9 2023 9:37AM,258995,10,Bio-PRF,15,
2,Mar 9 2023 9:18AM,258994,10,Eywa Pharma Inc.,1,
3,Mar 9 2023 9:18AM,258947,15,"Alliance Pharma, Inc.",36,
4,Mar 9 2023 9:17AM,258866,15,"Brookfield Pharmaceuticals, LLC",17,
5,Mar 9 2023 9:16AM,258993,22,"NBTY Global, Inc.",1,
6,Mar 9 2023 9:03AM,258990,10,ISDIN Corporation,1,
7,Mar 9 2023 9:01AM,258989,10,Methapharm-G,1,
8,Mar 9 2023 8:56AM,258988,19,"GCH Granules USA, Inc.",1,
9,Mar 9 2023 8:48AM,258987,10,"Digital Brands, LLC",4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:38AM,258996,15,"Alliance Pharma, Inc.",8,
1,Mar 9 2023 9:37AM,258995,10,Bio-PRF,15,
2,Mar 9 2023 9:18AM,258994,10,Eywa Pharma Inc.,1,
3,Mar 9 2023 9:18AM,258947,15,"Alliance Pharma, Inc.",36,
4,Mar 9 2023 9:17AM,258866,15,"Brookfield Pharmaceuticals, LLC",17,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:38AM,258996,15,"Alliance Pharma, Inc.",8,NaN
1,Mar 9 2023 9:37AM,258995,10,Bio-PRF,15,NaN
2,Mar 9 2023 9:18AM,258994,10,Eywa Pharma Inc.,1,NaN
3,Mar 9 2023 9:18AM,258947,15,"Alliance Pharma, Inc.",36,NaN
4,Mar 9 2023 9:17AM,258866,15,"Brookfield Pharmaceuticals, LLC",17,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 9 2023 9:38AM,258996,15,"Alliance Pharma, Inc.",8,0.0
1,Mar 9 2023 9:37AM,258995,10,Bio-PRF,15,0.0
2,Mar 9 2023 9:18AM,258994,10,Eywa Pharma Inc.,1,0.0
3,Mar 9 2023 9:18AM,258947,15,"Alliance Pharma, Inc.",36,0.0
4,Mar 9 2023 9:17AM,258866,15,"Brookfield Pharmaceuticals, LLC",17,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2071862,37,0.0
12,258985,1,0.0
15,776809,61,0.0
19,1294886,73,20.0
22,258993,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2071862,37,0.0
1,12,258985,1,0.0
2,15,776809,61,0.0
3,19,1294886,73,20.0
4,22,258993,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,37,0.0
1,12,1,0.0
2,15,61,0.0
3,19,73,20.0
4,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,12,Released,1.0
2,15,Released,61.0
3,19,Released,73.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,22
Status,,,,,
Executing,0.0,0.0,0.0,20.0,0.0
Released,37.0,1.0,61.0,73.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,22
0,Executing,0.0,0.0,0.0,20.0,0.0
1,Released,37.0,1.0,61.0,73.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,22
0,Executing,0.0,0.0,0.0,20.0,0.0
1,Released,37.0,1.0,61.0,73.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()